# Minatwo : Adversarial Detectors for Kitsune-based IDS


## Library Import

In [ ]:
%cd drive/MyDrive/Thesis/Utils/Kitsune/
!pip install scapy adversarial-robustness-toolbox >/dev/null

import zipfile
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle as pkl
import scipy.cluster.hierarchy as sch
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, CarliniL2Method, ProjectedGradientDescentTensorFlowV2, SaliencyMapMethod, DeepFool
from art.classifiers import TensorFlowV2Classifier

# Select the training device
device = '/GPU:0'

/content/drive/MyDrive/Thesis/Utils/Kitsune


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The module art.classifiers will be removed in ART 1.8.0 and replaced with art.estimators.classification
  # This is added back by InteractiveShellApp.init_path()


## NSL-KDD Pre-Processing

### Dowloading and importing the dataset

In [ ]:
#Downloading and extracting the dataset if it doesn't exist
!if [ ! -d "./NSL-KDD" ]; then wget http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip; mkdir NSL-KDD; unzip NSL-KDD.zip -d NSL-KDD; fi
    
#Importing the training and testing datasets from .CSV to Pandas DataFrames
features = ['1 Duration', '2 Protocol-type : ', '3 Service : ', '4 Flag : ', '5 Src-bytes', '6 Dst-bytes', '7 Land', '8 Wrong-fragment', '9 Urgent', '10 Hot', '11 Num-failed-logins', '12 Logged-in', '13 Num-compromised', '14 Root-shell', '15 Su-attempted', '16 Num-root', '17 Num-file-creations', '18 Num-shells', '19 Num-access-files', '20 Num-outbound-cmds', '21 Is-host-login', '22 Is-guest-login', '23 Count', '24 Srv-count', '25 Serror-rate', '26 Srv-serror-rate', '27 Rerror-rate', '28 Srv-rerror-rate', '29 Same-srv-rate', '30 Diff-srv-rate', '31 Srv-diff-host-rate', '32 Dst-host-count', '33 Dst-host-srv-count', '34 Dst-host-same-srv-rate', '35 Dst-host-diff-srv-rate', '36 Dst-host-same-src-port-rate', '37 Dst-host-srv-diff-host-rate', '38 Dst-host-serror-rate', '39 Dst-host-srv-serror-rate', '40 Dst-host-rerror-rate', '41 Dst-host-srv-rerror-rate', '42 Attack_type', '43 Difficulty']
df_training = pd.read_csv('./NSL-KDD/KDDTrain+_20Percent.txt', names=features)
df_testing = pd.read_csv('./NSL-KDD/KDDTest+.txt', names=features)
# Stack the training and test sets
data = pd.concat([df_training, df_testing], axis=0)

### Removing unused features

In [ ]:
# Drop the last column (which might be the difficulty, so it's useless)
data.drop('43 Difficulty', inplace=True, axis=1)
# Drop the 19th column wich is full of 0, so has std=0. which causes issues for the normalization
data.drop('20 Num-outbound-cmds', inplace=True, axis=1)

### Transforming the problem into binary clasification

In [ ]:
# Transform the nominal attribute "Attack type" into binary (0 : normal / 1 : attack)
labels = (data['42 Attack_type'] != 'normal').astype('int64')
data['42 Labels'] = labels
data.drop('42 Attack_type', inplace=True, axis=1)

### One Hot Encoding the categorical features

In [ ]:
# One Hot Encode the 3 first nominal attributes and drop them
for i in ['4 Flag : ', '3 Service : ', '2 Protocol-type : ']:
    # Create the One Hot Encode DataFrame
    dum = pd.get_dummies(data[i])
    # Insert into the dataset DataFrame by Series
    for column_name in list(dum.columns):
        data.insert(1, str(i)+column_name, dum[column_name])
        data[str(i)+column_name] = data[str(i)+column_name].astype('int64')
    # Drop the old attribute's column
    data.drop(i, inplace=True, axis=1)

### Spliting the training and test set 

In [ ]:
# Split training and test sets
df_training = data[:df_training.shape[0]]
df_testing = data[df_training.shape[0]:]

### Normalizing the data using Min-Max

In [ ]:
# Min-Max normalization on the non binary features
for i in ['1 Duration', '5 Src-bytes', '6 Dst-bytes', '8 Wrong-fragment', '9 Urgent', '10 Hot', '11 Num-failed-logins', '13 Num-compromised', '15 Su-attempted', '16 Num-root', '17 Num-file-creations', '18 Num-shells', '19 Num-access-files', '23 Count', '24 Srv-count', '25 Serror-rate', '26 Srv-serror-rate', '27 Rerror-rate', '28 Srv-rerror-rate', '29 Same-srv-rate', '30 Diff-srv-rate', '31 Srv-diff-host-rate', '32 Dst-host-count', '33 Dst-host-srv-count', '34 Dst-host-same-srv-rate', '35 Dst-host-diff-srv-rate', '36 Dst-host-same-src-port-rate', '37 Dst-host-srv-diff-host-rate', '38 Dst-host-serror-rate', '39 Dst-host-srv-serror-rate', '40 Dst-host-rerror-rate', '41 Dst-host-srv-rerror-rate']:
    # The min and max are only computed from the training set
    min_val = df_training[i].min()
    max_val = df_training[i].max()
    df_training[i] = ((df_training[i] - min_val) / (max_val - min_val)) 
    df_testing[i] = ((df_testing[i] - min_val) / (max_val - min_val)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Convert the training and testing set into NumPy array

In [ ]:
# Get NumPy arrays from DataFrames
nd_training = df_training.values
nd_testing = df_testing.values

### Extracting the labels and making copies

In [ ]:
# Separating arguments (x) from lables (y)
x_train = nd_training[:, :-1]
y_train = nd_training[:, -1]
x_test = nd_testing[:, :-1]
y_test = nd_testing[:, -1]

tr = 2000
te = 500
# Make a copy of the data set as NumPy arrays
x_train_np = x_train.copy()
y_train_np = y_train.copy()
x_test_np = x_test.copy()
y_test_np = y_test.copy()

## Feature Map

### Initial Feature Map

In [ ]:
max_clust = 10

corr = pd.DataFrame(x_train_np).corr().values
corr = np.nan_to_num(corr, nan=1)
dist = sch.distance.pdist(corr)
link = sch.linkage(dist, method='complete')
ind = sch.fcluster(link, max_clust, 'maxclust')

### Distribution

In [ ]:
feature_map = []
for cluster in range(max_clust):
  feature_map.append(np.where(ind == cluster+1)[0].tolist())

# Distribution
distribute = []
for index in range(len(feature_map)):
  if len(feature_map[index]) == 1:
    distribute.append((index, feature_map[index]))
for distrib in distribute:
  feature_map.pop(distrib[0])
  min(feature_map, key=len).append(distrib[1][0])

### Cut

In [ ]:
ordered = np.argsort(ind)
feature_n = len(ordered) // max_clust
rest = len(ordered) % max_clust
feature_map = []
i = 0
while i < len(ordered):
  elem = feature_n if rest < 0 else feature_n + 1
  rest -= 1
  feature_map.append(ordered[i:i+elem].tolist())
  i += elem

### Datasets

In [ ]:
train_data = x_train_np.copy()
y_train_ohe = pd.get_dummies(y_train_np)
datasets_train = []
for feature_list in feature_map:
  sub_data = []
  for row_index in range(len(train_data)):
    sub_data.append([train_data[row_index][feature] for feature in feature_list])
  datasets_train.append(sub_data)

for i in range(len(datasets_train)):
  with tf.device(device_name=device):
    datasets_train[i] = tf.convert_to_tensor(datasets_train[i], np.float32)
with tf.device(device_name=device):
  y_train_ts = tf.convert_to_tensor(y_train_np, np.float32)
  y_train_ohe = tf.convert_to_tensor(y_train_ohe, np.float32)

In [ ]:
test_data = x_test_np.copy()
y_test_ohe = pd.get_dummies(y_test_np)
datasets_test = []
for feature_list in feature_map:
  sub_data = []
  for row_index in range(len(test_data)):
    sub_data.append([test_data[row_index][feature] for feature in feature_list])
  datasets_test.append(sub_data)

for i in range(len(datasets_test)):
  with tf.device(device_name=device):
    datasets_test[i] = tf.convert_to_tensor(datasets_test[i], np.float32)
with tf.device(device_name=device):
  y_test_ts = tf.convert_to_tensor(y_test_np, np.float32)
  y_test_ohe = tf.convert_to_tensor(y_test_ohe, np.float32)

## Kitsutwo

### Design

In [ ]:
output_number = 2
eval_metric = 'categorical_accuracy'
loss_fn = tf.keras.losses.CategoricalCrossentropy()
activ_out = 'softmax'
neurons_number = 256
lr = 0.01

ensemble_layer = []
for i in range(len(datasets_train)):
  ae = tf.keras.Sequential([
      tf.keras.layers.Dense(neurons_number, input_shape=(len(feature_map[i]),), activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(output_number, activation=activ_out)
  ])
  ae.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
      loss=loss_fn,
      metrics=[eval_metric]
  )
  ensemble_layer.append(ae)

output_layer = tf.keras.Sequential([
      tf.keras.layers.Dense(neurons_number, input_shape=(len(ensemble_layer)*output_number,), activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(1, activation='sigmoid')
])
output_layer.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=['binary_accuracy']
)

### Ensemble Layer

In [ ]:
for index in range(len(ensemble_layer)):
  best_value = 0.0
  model = ensemble_layer[index]
  model_file = "./Ensemble/{}".format(index)

  # Iterrating on the dataset
  for i in range(3):
    model.fit(x=datasets_train[index], y=y_train_ohe, epochs=10, batch_size=100, verbose=0)
    eval = model.evaluate(x=datasets_test[index], y=y_test_ohe, verbose=1)
    eval_value = eval[-1]
    if eval_value > best_value:
      best_value = eval_value
      tf.keras.models.save_model(model, model_file)

In [ ]:
ensemble_layer = []
for i in range(len(feature_map)):
  ensemble_layer.append(tf.keras.models.load_model("./Ensemble/{}".format(i)))
  ensemble_layer[i].evaluate(x=datasets_test[i], y=y_test_ohe, verbose=2)

705/705 - 1s - loss: 0.7068 - categorical_accuracy: 0.4308
705/705 - 1s - loss: 0.8463 - categorical_accuracy: 0.7634
705/705 - 1s - loss: 1.4101 - categorical_accuracy: 0.6882
705/705 - 1s - loss: 0.6639 - categorical_accuracy: 0.6854
705/705 - 1s - loss: 0.6955 - categorical_accuracy: 0.7078
705/705 - 1s - loss: 0.5288 - categorical_accuracy: 0.7948
705/705 - 1s - loss: 0.7227 - categorical_accuracy: 0.4894
705/705 - 1s - loss: 0.7030 - categorical_accuracy: 0.4308
705/705 - 1s - loss: 2.1032 - categorical_accuracy: 0.5983


### Output Layer

In [ ]:
predictions_train = []
predictions_test = []
output_number = 2
for i in range(len(ensemble_layer)):
  pred_tr = ensemble_layer[i].predict(datasets_train[i])
  pred_te = ensemble_layer[i].predict(datasets_test[i])
  for j in range(output_number):
    predictions_train.append([p[j] for p in pred_tr])
    predictions_test.append([p[j] for p in pred_te])

output_dataset_train =[[predictions_train[j][i] for j in range(len(predictions_train))] for i in range(len(predictions_train[0]))]
output_dataset_test = [[predictions_test[j][i] for j in range(len(predictions_test))] for i in range(len(predictions_test[0]))]
with tf.device(device_name=device):
  output_dataset_train = tf.convert_to_tensor(output_dataset_train, np.float32)
  output_dataset_test = tf.convert_to_tensor(output_dataset_test, np.float32)

In [ ]:
best_value = 0.0
model_file = "./Output/"

# Iterrating on the dataset
for i in range(3):
  output_layer.fit(x=output_dataset_train, y=y_train_ts, epochs=10, batch_size=100, verbose=0)
  eval = output_layer.evaluate(x=output_dataset_test, y=y_test_ts, verbose=0)
  eval_value = eval[-1]
  if eval_value > best_value:
    best_value = eval_value
    tf.keras.models.save_model(output_layer, model_file)

INFO:tensorflow:Assets written to: ./Output/assets
INFO:tensorflow:Assets written to: ./Output/assets
INFO:tensorflow:Assets written to: ./Output/assets
705/705 [==============================] - 1s 1ms/step - loss: 0.8749 - binary_accuracy: 0.8158


[0.8749207258224487, 0.8158268332481384]

In [ ]:
output_layer = tf.keras.models.load_model("./Output/")
output_layer.evaluate(x=output_dataset_test, y=y_test_ts, verbose=1)

705/705 [==============================] - 1s 2ms/step - loss: 0.8749 - binary_accuracy: 0.8158


[0.8749207258224487, 0.8158268332481384]

### Total

In [ ]:
def get_accuracy(datasets, labels):
  predictions = []
  for i in range(len(ensemble_layer)):
    pred = ensemble_layer[i].predict(datasets[i])
    for j in range(output_number):
      predictions.append([p[j] for p in pred])

  output_dataset = [[predictions[j][i] for j in range(len(predictions))] for i in range(len(predictions[0]))]
  with tf.device(device_name=device):
    output_dataset = tf.convert_to_tensor(output_dataset, np.float32)
  eval = output_layer.evaluate(x=output_dataset, y=labels, verbose=0)
  return eval[-1]

In [ ]:
get_accuracy(datasets_test, y_test_ts)

0.8158268332481384

## Adversarial Attacks

### ART Classifiers

In [ ]:
lr = 0.1
loss_fn = tf.keras.losses.CategoricalCrossentropy()
classifiers = []
for index in range(len(ensemble_layer)):
  shape = len(feature_map[index])
  model = ensemble_layer[index]
  classifiers.append(TensorFlowV2Classifier(model=model, loss_object=loss_fn, input_shape=(shape,), nb_classes=2))

### FGSM

In [ ]:
adversarial_examples_train = []
adversarial_examples_test = []
for index in range(len(ensemble_layer)):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = FastGradientMethod(classifiers[index],
                                           norm=np.inf,
                                           eps=0.1,
                                           targeted=False,
                                           num_random_init=100,
                                           batch_size=128)
  
  # Generating the adversarial examples
  adversarial_examples_test.append(adversarial_crafter.generate(x=datasets_test[index], y=y_test_ohe))
  adversarial_examples_train.append(adversarial_crafter.generate(x=datasets_train[index], y=y_train_ohe))

# Saving adversarial examples
with open("FGSM_tr.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("FGSM_te.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test_ts)

Crafting examples for classifier 0
Crafting examples for classifier 1
Crafting examples for classifier 2
Crafting examples for classifier 3
Crafting examples for classifier 4
Crafting examples for classifier 5
Crafting examples for classifier 6
Crafting examples for classifier 7
Crafting examples for classifier 8


0.26290810108184814

### PGD

In [ ]:
adversarial_examples_train = []
adversarial_examples_test = []
for index in range(len(ensemble_layer)):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = ProjectedGradientDescentTensorFlowV2(classifiers[index],
                                                            norm=np.inf,
                                                            eps=0.1,
                                                            eps_step=0.001,
                                                            max_iter=100,
                                                            targeted=False,
                                                            batch_size=128,
                                                            verbose=True)
  
  # Generating the adversarial examples
  adversarial_examples_test.append(adversarial_crafter.generate(x=datasets_test[index], y=y_test_ohe))
  adversarial_examples_train.append(adversarial_crafter.generate(x=datasets_train[index], y=y_train_ohe))

# Saving adversarial examples
with open("PGD_tr.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("PGD_te.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test_ts)

Crafting examples for classifier 0


Crafting examples for classifier 1


Crafting examples for classifier 2


Crafting examples for classifier 3


Crafting examples for classifier 4


Crafting examples for classifier 5


Crafting examples for classifier 6


Crafting examples for classifier 7


Crafting examples for classifier 8


0.13200852274894714

### CW

In [ ]:
adversarial_examples_train = []
adversarial_examples_test = []
for index in range(len(ensemble_layer)):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = CarliniL2Method(classifiers[index],
                                        confidence=0.3,
                                        targeted=False,
                                        learning_rate=1,
                                        binary_search_steps=45,
                                        max_iter=10,
                                        initial_const=0.01,
                                        max_halving=5,
                                        max_doubling=5,
                                        batch_size=128)

  # Generating the adversarial examples
  adversarial_examples_test.append(adversarial_crafter.generate(x=datasets_test[index], y=y_test_ohe.numpy()))
  adversarial_examples_train.append(adversarial_crafter.generate(x=datasets_train[index], y=y_train_ohe.numpy()))

  # Saving adversarial examples
  with open("CW_tr.pkl", "wb") as f:
    pkl.dump(adversarial_examples_train, f)
  with open("CW_te.pkl", "wb") as f:
    pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test_ts)

Crafting examples for classifier 7


C&W L_2:   0%|          | 0/177 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/197 [00:00<?, ?it/s]

Crafting examples for classifier 8


C&W L_2:   0%|          | 0/177 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/197 [00:00<?, ?it/s]

ValueError: ignored

### DF

In [ ]:
adversarial_examples_train = []
adversarial_examples_test = []
for index in range(len(ensemble_layer)):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = DeepFool(classifiers[index],
                                epsilon=0.01,
                                batch_size=128)
  
  # Generating the adversarial examples
  adversarial_examples_test.append(adversarial_crafter.generate(x=datasets_test[index], y=y_test_ohe.numpy()))
  adversarial_examples_train.append(adversarial_crafter.generate(x=datasets_train[index], y=y_train_ohe.numpy()))

# Saving adversarial examples
with open("DF_tr_01.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("DF_te_01.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test_ts)

Crafting examples for classifier 0


DeepFool:   0%|          | 0/177 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/197 [00:00<?, ?it/s]

Crafting examples for classifier 1


DeepFool:   0%|          | 0/177 [00:00<?, ?it/s]

...

DeepFool:   0%|          | 0/197 [00:00<?, ?it/s]

Crafting examples for classifier 8


DeepFool:   0%|          | 0/177 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/197 [00:00<?, ?it/s]

0.5116660594940186

## Adversarial Datasets

### Loading adversarial examples

In [ ]:
path = "Adversarial/"

with open(path+"FGSM_tr.pkl", "rb") as f:
  fgsm_train = pkl.load(f)
with open(path+"FGSM_te.pkl", "rb") as f:
  fgsm_test = pkl.load(f)

with open(path+"PGD_tr.pkl", "rb") as f:
  pgd_train = pkl.load(f)
with open(path+"PGD_te.pkl", "rb") as f:
  pgd_test = pkl.load(f)

with open(path+"CW_tr.pkl", "rb") as f:
  cw_train = pkl.load(f)
with open(path+"CW_te.pkl", "rb") as f:
  cw_test = pkl.load(f)

with open(path+"DF_tr.pkl", "rb") as f:
  df_train = pkl.load(f)
with open(path+"DF_te.pkl", "rb") as f:
  df_test = pkl.load(f)

### Crafting datasets

In [ ]:
x_advs_train = []
x_advs_test = []
y_advs_train = []
y_advs_test = []
for index in range(len(feature_map)):
  labels_0_tr = [0 for _ in range(datasets_train[index].shape[0])]
  labels_0_te = [0 for _ in range(datasets_test[index].shape[0])]
  labels_1_tr = [1 for _ in range(datasets_train[index].shape[0])]
  labels_1_te = [1 for _ in range(datasets_test[index].shape[0])]

  # Clean examples with label 0
  clean_train = pd.DataFrame(datasets_train[index].numpy())
  clean_train[clean_train.shape[1]] = labels_0_tr
  clean_test = pd.DataFrame(datasets_test[index].numpy())
  clean_test[clean_test.shape[1]] = labels_0_te

  # Altered examples with label 1
  fgsm_train_adv = pd.DataFrame(fgsm_train[index])
  fgsm_train_adv[fgsm_train_adv.shape[1]] = labels_1_tr
  fgsm_test_adv = pd.DataFrame(fgsm_test[index])
  fgsm_test_adv[fgsm_test_adv.shape[1]] = labels_1_te

  pgd_train_adv = pd.DataFrame(pgd_train[index])
  pgd_train_adv[pgd_train_adv.shape[1]] = labels_1_tr
  pgd_test_adv = pd.DataFrame(pgd_test[index])
  pgd_test_adv[pgd_test_adv.shape[1]] = labels_1_te

  cw_train_adv = pd.DataFrame(cw_train[index])
  cw_train_adv[cw_train_adv.shape[1]] = labels_1_tr
  cw_test_adv = pd.DataFrame(cw_test[index])
  cw_test_adv[cw_test_adv.shape[1]] = labels_1_te

  df_train_adv = pd.DataFrame(df_train[index])
  df_train_adv[df_train_adv.shape[1]] = labels_1_tr
  df_test_adv = pd.DataFrame(df_test[index])
  df_test_adv[df_test_adv.shape[1]] = labels_1_te

  size_quarter = clean_train[0].shape[0] // 4
  balanced_train_adv = pd.DataFrame().append([
      fgsm_train_adv.iloc[:size_quarter],
      pgd_train_adv.iloc[size_quarter:2*size_quarter],
      cw_train_adv.iloc[2*size_quarter:3*size_quarter],
      df_train_adv.iloc[3*size_quarter:],
  ])
  size_quarter = clean_test[0].shape[0] // 4
  balanced_test_adv = pd.DataFrame().append([
      fgsm_test_adv.iloc[:size_quarter],
      pgd_test_adv.iloc[size_quarter:2*size_quarter],
      cw_test_adv.iloc[2*size_quarter:3*size_quarter],
      df_test_adv.iloc[3*size_quarter:],
  ])
  
  adversarial_dataset_train = clean_train.append(cw_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(df_test_adv, ignore_index=True)
  x_advs_train.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train.append(adversarial_dataset_train.values[:, -1])
  y_advs_test.append(adversarial_dataset_test.values[:, -1])

## Adversarial Detectors

### Train ADD

In [ ]:
detectors = ensemble_layer.copy()

# General parameters
eval_metric = 'binary_accuracy'
lr = 0.01
optim = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.BinaryCrossentropy()
neurons = 256

for detector in detectors: # Add the ReLu layers
  detector.pop()
  detector.add(tf.keras.layers.Dense(neurons, activation="relu"))
  detector.add(tf.keras.layers.Dense(neurons, activation="relu"))
  detector.add(tf.keras.layers.Dense(1, activation="sigmoid"))
  detector.compile(optimizer=optim, loss=loss_fn, metrics=[eval_metric])

In [ ]:
for index in range(len(detectors)):
  best_value = 0.0
  detector = detectors[index]
  det_file = "./ADD/{}".format(index)

  # Iterrating on the dataset
  for i in range(3):
    detector.fit(x=x_advs_train[index], y=y_advs_train[index], epochs=10, batch_size=100, verbose=0)
    eval = detector.evaluate(x_advs_test[index], y_advs_test[index], verbose=0)
    eval_value = eval[-1]
    if eval_value > best_value:
      best_value = eval_value
      tf.keras.models.save_model(detector, det_file)
      print("Detector {} : new best score of {}".format(index, best_value))

INFO:tensorflow:Assets written to: ./ADD/0/assets
Detector 0 : new best score of 0.5
INFO:tensorflow:Assets written to: ./ADD/1/assets
Detector 1 : new best score of 0.6661861538887024
INFO:tensorflow:Assets written to: ./ADD/1/assets
Detector 1 : new best score of 0.6848828792572021
INFO:tensorflow:Assets written to: ./ADD/2/assets
Detector 2 : new best score of 0.7663457989692688
INFO:tensorflow:Assets written to: ./ADD/2/assets
Detector 2 : new best score of 0.7718461751937866
INFO:tensorflow:Assets written to: ./ADD/2/assets
Detector 2 : new best score of 0.776902973651886
INFO:tensorflow:Assets written to: ./ADD/3/assets
Detector 3 : new best score of 0.7335432767868042
INFO:tensorflow:Assets written to: ./ADD/3/assets
Detector 3 : new best score of 0.7345191836357117
INFO:tensorflow:Assets written to: ./ADD/4/assets
Detector 4 : new best score of 0.5
INFO:tensorflow:Assets written to: ./ADD/5/assets
Detector 5 : new best score of 0.8028300404548645
INFO:tensorflow:Assets written 

### Evaluate ADD

In [ ]:
detectors = []
for index in range(len(feature_map)):
  detector = tf.keras.models.load_model("./ADD/{}".format(index))
  detectors.append(detector)
  eval = detector.evaluate(x_advs_test[index], y_advs_test[index], verbose=0)[-1]
  print("Detector {} : {}".format(index, eval))

Detector 0 : 0.5
Detector 1 : 0.8964247703552246
Detector 2 : 0.8271380662918091
Detector 3 : 0.6009137630462646
Detector 4 : 0.5208037495613098
Detector 5 : 0.7596256136894226
Detector 6 : 0.48061567544937134
Detector 7 : 0.5
Detector 8 : 0.5


## Fusion Rules

### Definition

#### Majority Vote

In [ ]:
def MajorityVoteRule(ClassProbabilities):
  ClassFrequency = np.argmax(ClassProbabilities, axis=1) 
  counts = np.bincount(ClassFrequency)
  MajorityVotingClass = np.argmax(counts)
  return MajorityVotingClass

#### Simple Bayes Average

In [ ]:
def SimpleBayesAverageFusion(ClassProbabilities):
  SimpleBayesAverage=sum(ClassProbabilities,0)/ClassProbabilities.shape[0]
  if SimpleBayesAverage[0]>=SimpleBayesAverage[1]:
    return 0,SimpleBayesAverage[0]
  return 1,SimpleBayesAverage[1]

#### Dempster Shafer Evidence Combination

In [ ]:
def DempsterRule(m1, m2):
    ## extract the frame of discernment      
    sets=set(m1.keys()).union(set(m2.keys()))
    result=dict.fromkeys(sets,0)
    ## Combination process
    for i in m1.keys():
        for j in m2.keys():
            if set(str(i)).intersection(set(str(j))) == set(str(i)):
                result[i]+=m1[i]*m2[j]
            elif set(str(i)).intersection(set(str(j))) == set(str(j)):
                result[j]+=m1[i]*m2[j]
                 
    ## normalize the results
    f= sum(list(result.values()))
    for i in result.keys():
        result[i] /=f
    return result

def DempsterShaferEvidenceCombinationRule (ClassProbabilities, Weights, DontKnowClass=False):
  MassFunctionList= []
  for i in range(ClassProbabilities.shape[0]):
    MassFunction = {"a":Weights[i,0]*ClassProbabilities[i,0], "b":Weights[i,1]*ClassProbabilities[i,1],"ab":1-Weights[i,0]*ClassProbabilities[i,0]-Weights[i,1]*ClassProbabilities[i,1]}
    MassFunctionList.append(MassFunction)
    
  for i,mass in enumerate(MassFunctionList):
    if i == 0:
      DSresult = mass
    else: 
      DSresult = DempsterRule(DSresult,mass)

  if DSresult['a'] >= DSresult['b']:
    return 0, DSresult['a']
  else:
    return 1, DSresult['b'] 

### Compute results

#### Turn mono-neuronal output into bi-class probabilities

In [ ]:
probas = []
for index in range(len(detectors)):
  detector = detectors[index]
  pred = detector.predict(x_advs_test[index])
  proba = []
  for i in pred:
    proba.append([1-i[0], i[0]])
  probas.append(proba)

#### Majority Vote Rule

In [ ]:
guesses = []
for sample in range(len(probas[0])):
  class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
  guesses.append(MajorityVoteRule(class_proba))

correct = y_advs_test[0]
hit = 0
for i in range(len(guesses)):
  if guesses[i] == correct[i]:
    hit += 1
acc = hit / len(guesses)
print(acc)

0.5950363733144074


#### Simple Bayes Average

In [ ]:
guesses = []
for sample in range(len(probas[0])):
  class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
  guesses.append(SimpleBayesAverageFusion(class_proba))

correct = y_advs_test[0]
hit = 0
for i in range(len(guesses)):
  if guesses[i][0] == correct[i]:
    hit += 1
acc = hit / len(guesses)
print(acc)

0.8901703335699077


#### Dempster Shafer Evidence Combination

In [ ]:
# Get the weights
weights = []
correct = y_advs_test[0]
amount_1 = np.count_nonzero(correct == 1)
amount_0 = np.count_nonzero(correct == 0)
for proba in probas:
  weight = []
  hit_0 = 0
  hit_1 = 0
  for i in range(len(proba)):
    if proba[i].index(max(proba[i])) == correct[i]:
      if correct[i]:
        hit_1 += 1
      else:
        hit_0 += 1
  weight = [hit_0/amount_0, hit_1/amount_1]
  weights.append(weight)
weights = np.array(weights)

guesses = []
for sample in range(len(probas[0])):
  class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
  guesses.append(DempsterShaferEvidenceCombinationRule(class_proba, weights))

hit = 0
for i in range(len(guesses)):
  if guesses[i][0] == correct[i]:
    hit += 1
acc = hit / len(guesses)
print(acc)

0.6715090489709014
